In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
os.chdir("/content/")
!ls

drive  sample_data


In [5]:

! pip install ftfy
! pip install emoji
! pip install fasttext
! pip install langdetect
!pip -q install ftfy emoji contractions spacy tqdm
!python -m spacy download en_core_web_sm

! pip install spacy-transformers
! pip install benepar
! pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 125.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip -q install scispacy  # or without [tensorflow] if using CPU only


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 49.1 MB/s eta 0:00:00


In [4]:
!pip -q install ftfy emoji contractions spacy sentencepiece transformers
!python -m spacy download en_core_web_sm
# Optional: SciSpaCy for symptom entities  ⇢ comment out if you’ll add later
# !pip -q install scispacy
# !pip -q install https://s3.amazonaws.com/allenai-scispacy/models/en_ner_bc5cdr_md-0.5.3.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en_core_web_sm 3.8.0
    Uninstalling en_core_web_sm-3.8.0:
      Successfully uninstalled en_core_web_sm-3.8.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
import pandas as pd
from pathlib import Path
from html import unescape            # decode &amp; → &
from urllib.parse import unquote     # decode %20   → space
from ftfy import fix_text
import re, emoji, spacy, nltk
from collections import defaultdict
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import plotly.express as px
from wordcloud import WordCloud
from textblob import TextBlob
import fasttext
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
import warnings


import re, html, urllib.parse, string, itertools, collections, textwrap
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import ftfy, emoji, contractions, spacy

from spacy import displacy
warnings.filterwarnings('ignore')






## clean_reddit_csv.py

• Read the Reddit CSV   reddit_supplements.new
• Decode HTTP-/HTML-encoded characters in textual columns
• Drop duplicate rows (exact duplicates)
• Convert 'created_utc' (Unix seconds) to timezone-aware datetime
• Locate rows whose text contains "[removed]"
• Save two outputs:
    ─ cleaned_reddit.csv         (full cleaned data, duplicates removed)
    ─ removed_rows.csv           (only the '[removed]' rows for inspection)


In [8]:

nlp = spacy.load('en_core_web_sm')
data = pd.read_csv("/content/drive/MyDrive/NLP/data/supplements.csv", dtype=str)
data.head()

,type,id,parent_id,score,title,upvote_ratio,comment_depth,text,author,created_utc,num_comments,flair,url,date_time
0,submission,1kv9962,NaN,1,What do you think about these AI suggestions?,1.0,NaN,"\n\n*""What you call 'mental illness' may be yo...",Low_Appointment_3917,1748197135.0,1,NaN,https://www.reddit.com/r/Supplements/comments/...,2025-05-25 18:18:55+00:00
1,submission,1kv92e2,NaN,3,My tongkat ali experience.,1.0,NaN,"Long story short. I am a 25 yr old male, who h...",solama_Official,1748196655.0,8,Recommendations,https://www.reddit.com/r/Supplements/comments/...,2025-05-25 18:10:55+00:00
2,comment,mu7mbap,1kv92e2,1,NaN,NaN,0,Glad you found a reasonable remedy! 😎,DanHarderEHS4,1748196917.0,NaN,NaN,NaN,2025-05-25 18:15:17+00:00
3,comment,mu7mojv,1kv92e2,1,NaN,NaN,0,25 year old male 300 is low,c07e,1748197032.0,NaN,NaN,NaN,2025-05-25 18:17:12+00:00
4,comment,mu7mk2u,1kv92e2,1,NaN,NaN,0,"P.S - my only side effects was diarrhea, and n...",solama_Official,1748196993.0,NaN,NaN,NaN,2025-05-25 18:16:33+00:00


# Language filtering


In [9]:
# Load spaCy model
nlp = spacy.load('en_core_web_sm')

# Read data
data = pd.read_csv("/content/drive/MyDrive/NLP/data/supplements.csv", dtype=str)

print(f"Original data shape: {data.shape}")
print(f"Original number of rows: {len(data)}")

# Function to detect if text is English
def is_english(text):
    """
    Detect if text is in English using langdetect library
    Returns True if English, False otherwise
    """
    if pd.isna(text) or text.strip() == '':
        return False

    try:
        # Detect language
        detected_lang = detect(text)
        return detected_lang == 'en'
    except LangDetectException:
        # If detection fails, assume it's not English
        return False

# Step 1: Remove rows with very short or empty text (less than 15 characters)
print("\nStep 1: Filtering out very short texts...")
data_filtered = data[data["text"].str.len() >= 15].copy()
print(f"After removing short texts: {len(data_filtered)} rows")

# Step 2: Filter only English language texts
print("\nStep 2: Filtering for English language only...")
# Apply language detection
data_filtered['is_english'] = data_filtered["text"].apply(is_english)

# Keep only English texts
data_english = data_filtered[data_filtered['is_english'] == True].copy()

# Drop the helper column
data_english = data_english.drop('is_english', axis=1)

print(f"After English filtering: {len(data_english)} rows")
print(f"Removed {len(data) - len(data_english)} rows in total")

# Display sample of filtered data
print("\nSample of filtered data:")
print(data_english.head())

# Optional: Save the filtered data
data_english.to_csv("/content/drive/MyDrive/NLP/data/supplements_old_filtered.csv", index=False)
print("\nFiltered data saved to supplements_old_filtered.csv")

Original data shape: (12388, 14)
Original number of rows: 12388

Step 1: Filtering out very short texts...
After removing short texts: 11683 rows

Step 2: Filtering for English language only...
After English filtering: 11243 rows
Removed 1145 rows in total

Sample of filtered data:
         type       id parent_id score  \
0  submission  1kv9962       NaN     1   
1  submission  1kv92e2       NaN     3   
2     comment  mu7mbap   1kv92e2     1   
3     comment  mu7mojv   1kv92e2     1   
4     comment  mu7mk2u   1kv92e2     1   

                                           title upvote_ratio comment_depth  \
0  What do you think about these AI suggestions?          1.0           NaN   
1                     My tongkat ali experience.          1.0           NaN   
2                                            NaN          NaN             0   
3                                            NaN          NaN             0   
4                                            NaN          NaN        

# cleaning processing tokenisation normalisation

In [11]:
CSV_PATH = Path("/content/drive/MyDrive/NLP/data/supplements_old_filtered.csv")

# Load as strings to avoid dtype surprises
df = pd.read_csv(CSV_PATH, dtype=str, on_bad_lines="skip")
print(f"Loaded {len(df):,} rows.")


Loaded 11,243 rows.


## TextCleaner

In [12]:
class TextCleaner:
    """HTML / URL / email / phone / contraction / whitespace cleaning"""

    _URL_RE   = re.compile(r"https?://\S+|www\.\S+", re.I)
    _EMAIL_RE = re.compile(r"\b[\w\.-]+@[\w\.-]+\.\w+\b")
    _PHONE_RE = re.compile(r"\b(\+?\d[\d\- ]{7,}\d)\b")
    _TAG_RE   = re.compile(r"<[^>]+>")

    def __init__(self, keep_punct=False):
        self.keep_punct = keep_punct

    def __call__(self, txt: str) -> str:
        if not isinstance(txt, str):
            return ""
        txt = ftfy.fix_text(txt)                       # 1) fix mojibake
        txt = urllib.parse.unquote(txt)                # 2) %20 → space
        txt = html.unescape(txt)                       # 3) &amp; → &
        txt = self._TAG_RE.sub(" ", txt)               # 4) strip HTML
        txt = self._URL_RE.sub(" ", txt)               # 5) strip URLs
        txt = self._EMAIL_RE.sub(" ", txt)             # 6) emails
        txt = self._PHONE_RE.sub(" ", txt)             # 7) phones
        txt = contractions.fix(txt)                    # 8) expand don't→do not
        txt = emoji.replace_emoji(txt, replace="")     # 9) remove emoji
        if not self.keep_punct:
            txt = txt.translate(str.maketrans("", "", string.punctuation))
        txt = re.sub(r"\s+", " ", txt)                 # 10) normalise WS
        return txt.strip().lower()


## TextTokenizer

In [13]:
class TextTokenizer:
    """
    • Keeps tokens that are NOUN / VERB / ADJ / ADV
    • Removes stopwords & very short tokens
    • Lemmatizes
    • Extracts named entities
    """
    KEEP_POS = {"NOUN", "VERB", "ADJ", "ADV"}

    def __init__(self, min_len=3):
        self.min_len = min_len
        self.nlp = spacy.load("en_core_web_sm", disable=["parser"])
        # speed-up: remove stopwords once
        self.stop = self.nlp.Defaults.stop_words

    def process(self, text):
        doc = self.nlp(text)
        tokens = [
            t.lemma_ for t in doc
            if (t.pos_ in self.KEEP_POS
                and not t.is_stop
                and len(t) >= self.min_len
                and t.is_alpha)
        ]
        ents = [(e.text, e.label_) for e in doc.ents]
        return " ".join(tokens), ents


## Apply cleaning + tokenisation with progress bar

In [14]:
cleaner   = TextCleaner()
tokeniser = TextTokenizer(min_len=3)

clean_texts   = []
lemmas        = []
entities_list = []

for txt in tqdm(df["text"].fillna("").tolist(), desc="Processing"):
    c = cleaner(txt)
    lem, ents = tokeniser.process(c)
    clean_texts.append(c)
    lemmas.append(lem)
    entities_list.append(ents)

df["clean_text"] = clean_texts
df["lemmas"]     = lemmas
df["entities"]   = entities_list


Processing:   0%|          | 0/11243 [00:00<?, ?it/s]

## Quality control

In [15]:
# Remove rows whose lemmas got too short (<3 words)
min_words = 5
long_mask = df["lemmas"].str.split().str.len() >= min_words
print(f"Dropping {len(df) - long_mask.sum():,} rows with <{min_words} tokens.")
df = df[long_mask].reset_index(drop=True)


Dropping 2,533 rows with <5 tokens.


In [16]:
# Quick stats
tok_counts = df["lemmas"].str.split().str.len()
print(tok_counts.describe())

common = collections.Counter(itertools.chain.from_iterable(
            s.split() for s in df["lemmas"])).most_common(20)
print("Top-20 tokens:", common)


count    8710.000000
mean       23.623421
std        34.816635
min         5.000000
25%         8.000000
50%        14.000000
75%        27.000000
max      1429.000000
Name: lemmas, dtype: float64
Top-20 tokens: [('supplement', 2893), ('take', 2652), ('vitamin', 1929), ('magnesium', 1728), ('good', 1695), ('day', 1645), ('help', 1643), ('feel', 1607), ('try', 1515), ('work', 1304), ('time', 1192), ('people', 1155), ('sleep', 1115), ('start', 1098), ('know', 1098), ('need', 1085), ('thing', 1043), ('think', 1028), ('get', 1018), ('effect', 972)]


In [17]:
# Sample before / after
for i in np.random.choice(df.index, 3, replace=False):
    print("-"*60)
    print("ORIG:", textwrap.shorten(df.loc[i,"text"], 200))
    print("CLEAN:", df.loc[i,"clean_text"][:200])
    print("LEMMA:", df.loc[i,"lemmas"][:200])


------------------------------------------------------------
ORIG: I started taking creatine every day last week. Today i am very itchy, especially on my face. Does this have to do with the creatine? Is it because i possibly don't drink enough water? Also i [...]
CLEAN: i started taking creatine every day last week today i am very itchy especially on my face does this have to do with the creatine is it because i possibly do not drink enough water also i want to menti
LEMMA: start take creatine day week today itchy especially face creatine possibly drink water want mention take creatine experience feedback highly appreciated
------------------------------------------------------------
ORIG: I used to think all this anti aging supplement stuff was just marketing hype but I'm hitting 40 soon and starting to feel it energy's dipping, recovery is slower and my focus isn't what it used [...]
CLEAN: i used to think all this anti aging supplement stuff was just marketing hype but i am hitting 

## Save processed data

In [18]:
OUT = Path("/content/drive/MyDrive/NLP/data/supplements_processed.parquet")
df.to_parquet(OUT, index=False)
print("Saved to", OUT)


Saved to /content/drive/MyDrive/NLP/data/supplements_processed.parquet


#Adding supplement terms

### Glossary

In [25]:
SUPP_ABBR_DICT = {
    # Original entries
    # hormones / pro-hormones
    "dhea": "dehydroepiandrosterone",
    "t": "testosterone",
    # cruciferous-vegetable compounds
    "dim": "diindolylmethane",
    "i3c": "indole-3-carbinol",
    # aminos & performance
    "bcaa": "branched-chain amino acids",
    "eaa": "essential amino acids",
    "hmb": "beta-hydroxy beta-methylbutyrate",
    "l-carn": "l-carnitine",
    # omega-3 & lipids
    "epa": "eicosapentaenoic acid",
    "dha": "docosahexaenoic acid",
    "ala": "alpha-linolenic acid",
    "cla": "conjugated linoleic acid",
    # botanicals
    "ashwagandha": "withania somnifera",
    "rhodiola": "rhodiola rosea",
    "tongkat": "eurycoma longifolia",
    "shilajit": "shilajit (mineral pitch)",
    # nootropics / misc.
    "nac": "n-acetyl cysteine",
    "coq10": "ubiquinone",
    "ala-acid": "alpha-lipoic acid",
    "cdp": "citicoline",
    "5-htp": "5-hydroxytryptophan",
    # dosage / units
    "mg": "milligram",
    "mcg": "microgram",
    "iu": "international unit",

    # EXPANDED ENTRIES

    # === VITAMINS ===
    # Fat-soluble vitamins
    "vit a": "vitamin a (retinol)",
    "vit d": "vitamin d (cholecalciferol)",
    "vit d2": "vitamin d2 (ergocalciferol)",
    "vit d3": "vitamin d3 (cholecalciferol)",
    "vit e": "vitamin e (tocopherol)",
    "vit k": "vitamin k (phylloquinone)",
    "vit k1": "vitamin k1 (phylloquinone)",
    "vit k2": "vitamin k2 (menaquinone)",

    # B-Complex vitamins
    "vit b": "b-complex vitamins",
    "b1": "thiamine (vitamin b1)",
    "b2": "riboflavin (vitamin b2)",
    "b3": "niacin (vitamin b3)",
    "b5": "pantothenic acid (vitamin b5)",
    "b6": "pyridoxine (vitamin b6)",
    "b7": "biotin (vitamin b7)",
    "b9": "folate (vitamin b9)",
    "b12": "cobalamin (vitamin b12)",
    "vit c": "ascorbic acid (vitamin c)",
    "folate": "folic acid (vitamin b9)",
    "biotin": "vitamin b7",
    "choline": "choline bitartrate",
    "inositol": "myo-inositol",
    "paba": "para-aminobenzoic acid",

    # === MINERALS ===
    "ca": "calcium",
    "mg-min": "magnesium",
    "fe": "iron",
    "zn": "zinc",
    "cu": "copper",
    "mn": "manganese",
    "se": "selenium",
    "cr": "chromium",
    "mo": "molybdenum",
    "i": "iodine",
    "k": "potassium",
    "na": "sodium",
    "p": "phosphorus",
    "s": "sulfur",
    "cl": "chloride",
    "v": "vanadium",
    "ni": "nickel",
    "si": "silicon",
    "bo": "boron",
    "co": "cobalt",

    # === AMINO ACIDS ===
    "arg": "l-arginine",
    "gln": "l-glutamine",
    "gly": "glycine",
    "tau": "taurine",
    "try": "l-tryptophan",
    "tyr": "l-tyrosine",
    "phe": "l-phenylalanine",
    "met": "l-methionine",
    "cys": "l-cysteine",
    "his": "l-histidine",
    "lys": "l-lysine",
    "thr": "l-threonine",
    "val": "l-valine",
    "leu": "l-leucine",
    "ile": "l-isoleucine",
    "pro": "l-proline",
    "ser": "l-serine",
    "ala-aa": "l-alanine",
    "asp": "l-aspartic acid",
    "glu": "l-glutamic acid",
    "orn": "l-ornithine",
    "cit": "l-citrulline",
    "beta-ala": "beta-alanine",
    "car": "l-carnitine",
    "acetyl-car": "acetyl-l-carnitine",
    "propionyl-car": "propionyl-l-carnitine",

    # === PERFORMANCE & FITNESS ===
    "crea": "creatine monohydrate",
    "cm": "creatine monohydrate",
    "hcl": "creatine hydrochloride",
    "kre": "kre-alkalyn (buffered creatine)",
    "pre": "pre-workout supplement",
    "pwo": "pre-workout supplement",
    "post": "post-workout supplement",
    "intra": "intra-workout supplement",
    "pump": "nitric oxide booster",
    "no": "nitric oxide",
    "no2": "nitric oxide booster",
    "wpi": "whey protein isolate",
    "wpc": "whey protein concentrate",
    "wph": "whey protein hydrolysate",
    "casein": "casein protein",
    "egg": "egg protein",
    "soy": "soy protein",
    "pea": "pea protein",
    "hemp": "hemp protein",
    "rice": "rice protein",
    "mass": "mass gainer",
    "gainer": "weight gainer",

    # === NOOTROPICS & BRAIN HEALTH ===
    "racetam": "racetam family nootropics",
    "pira": "piracetam",
    "oxi": "oxiracetam",
    "ani": "aniracetam",
    "phen": "phenylpiracetam",
    "mod": "modafinil",
    "adra": "adrafinil",
    "lion": "lion's mane mushroom",
    "bacopa": "bacopa monnieri",
    "ginkgo": "ginkgo biloba",
    "ps": "phosphatidylserine",
    "pc": "phosphatidylcholine",
    "dmae": "dimethylaminoethanol",
    "gpc": "alpha-gpc (glycerophosphocholine)",
    "pqq": "pyrroloquinoline quinone",
    "nad": "nicotinamide adenine dinucleotide",
    "nmn": "nicotinamide mononucleotide",
    "nr": "nicotinamide riboside",

    # === ADAPTOGENS & HERBS ===
    "gin": "panax ginseng",
    "ginseng": "panax ginseng",
    "sib": "siberian ginseng (eleuthero)",
    "maca": "lepidium meyenii",
    "tribulus": "tribulus terrestris",
    "fenu": "fenugreek",
    "saw": "saw palmetto",
    "milk": "milk thistle",
    "echi": "echinacea",
    "valerian": "valeriana officinalis",
    "passionflower": "passiflora incarnata",
    "kava": "piper methysticum",
    "st-john": "st. john's wort",
    "turmeric": "curcuma longa",
    "curc": "curcumin",
    "green-tea": "camellia sinensis",
    "egcg": "epigallocatechin gallate",
    "grape": "grape seed extract",
    "resveratrol": "trans-resveratrol",
    "quercetin": "quercetin",
    "rutin": "rutin",
    "hesperidin": "hesperidin",
    "bioflavonoids": "citrus bioflavonoids",

    # === DIGESTIVE & GUT HEALTH ===
    "prob": "probiotics",
    "pre-bio": "prebiotics",
    "fiber": "dietary fiber",
    "psyllium": "psyllium husk",
    "digest": "digestive enzymes",
    "amylase": "amylase enzyme",
    "protease": "protease enzyme",
    "lipase": "lipase enzyme",
    "lactase": "lactase enzyme",
    "betaine": "betaine hydrochloride",
    "hcl-sup": "hydrochloric acid supplement",
    "ox-bile": "ox bile extract",
    "pancreatin": "pancreatic enzymes",
    "bromelain": "bromelain enzyme",
    "papain": "papain enzyme",

    # === ANTIOXIDANTS ===
    "sod": "superoxide dismutase",
    "cat": "catalase",
    "gsh": "glutathione",
    "reduced-gsh": "reduced glutathione",
    "liposomal": "liposomal delivery system",
    "astaxanthin": "astaxanthin",
    "lycopene": "lycopene",
    "lutein": "lutein",
    "zeaxanthin": "zeaxanthin",
    "beta-carotene": "beta-carotene",
    "mixed-toco": "mixed tocopherols",
    "gamma-e": "gamma-tocopherol",
    "selenium": "selenomethionine",

    # === SLEEP & RECOVERY ===
    "mel": "melatonin",
    "gaba": "gamma-aminobutyric acid",
    "theanine": "l-theanine",
    "mag-gly": "magnesium glycinate",
    "mag-thr": "magnesium threonate",
    "mag-mal": "magnesium malate",
    "mag-cit": "magnesium citrate",
    "mag-ox": "magnesium oxide",
    "zma": "zinc magnesium aspartate",
    "montmorency": "montmorency cherry extract",
    "chamomile": "matricaria chamomilla",
    "lemon-balm": "melissa officinalis",

    # === JOINT & BONE HEALTH ===
    "gluc": "glucosamine",
    "chond": "chondroitin sulfate",
    "msm": "methylsulfonylmethane",
    "hyaluronic": "hyaluronic acid",
    "collagen": "hydrolyzed collagen",
    "bone-broth": "bone broth protein",
    "cal-mag": "calcium magnesium",
    "d3-k2": "vitamin d3 with k2",
    "strontium": "strontium citrate",
    "ipriflavone": "ipriflavone",
    "horsetail": "equisetum arvense",

    # === FAT BURNERS & WEIGHT MANAGEMENT ===
    "caf": "caffeine anhydrous",
    "caff": "caffeine",
    "green-coffee": "green coffee bean extract",
    "garcinia": "garcinia cambogia",
    "hca": "hydroxycitric acid",
    "forskolin": "coleus forskohlii",
    "yohimbe": "yohimbine hydrochloride",
    "synephrine": "bitter orange extract",
    "capsaicin": "capsicum extract",
    "cayenne": "cayenne pepper extract",
    "chromium-pic": "chromium picolinate",
    "white-bean": "white kidney bean extract",
    "chitosan": "chitosan fiber",
    "glucomannan": "konjac root fiber",
    "hoodia": "hoodia gordonii",
    "fucoxanthin": "brown seaweed extract",

    # === ENERGY & METABOLISM ===
    "b-complex": "b-vitamin complex",
    "ribose": "d-ribose",
    "cordyceps": "cordyceps sinensis",
    "rhodiola": "rhodiola rosea",
    "schisandra": "schisandra chinensis",
    "gotu-kola": "centella asiatica",
    "yerba-mate": "ilex paraguariensis",
    "guarana": "paullinia cupana",
    "kola-nut": "cola nitida",
    "theacrine": "teacrine",
    "dynamine": "methylliberine",

    # === HEART & CIRCULATION ===
    "hawthorne": "crataegus species",
    "garlic": "allicin extract",
    "red-yeast": "red yeast rice",
    "plant-sterols": "phytosterols",
    "policosanol": "policosanol",
    "nattokinase": "nattokinase enzyme",
    "serrapeptase": "serrapeptase enzyme",
    "lumbrokinase": "lumbrokinase enzyme",
    "horse-chestnut": "aesculus hippocastanum",
    "butcher-broom": "ruscus aculeatus",

    # === IMMUNE SUPPORT ===
    "vit-c": "ascorbic acid",
    "ester-c": "calcium ascorbate",
    "buffered-c": "buffered vitamin c",
    "zinc-pic": "zinc picolinate",
    "zinc-bis": "zinc bisglycinate",
    "elderberry": "sambucus nigra",
    "astragalus": "astragalus membranaceus",
    "reishi": "ganoderma lucidum",
    "shiitake": "lentinula edodes",
    "maitake": "grifola frondosa",
    "beta-glucan": "beta-1,3-glucan",
    "transfer-factor": "transfer factor",
    "colostrum": "bovine colostrum",

    # === SPECIALTY COMPOUNDS ===
    "phos": "phosphatidylserine",
    "dmg": "dimethylglycine",
    "tmg": "trimethylglycine (betaine)",
    "sam-e": "s-adenosyl methionine",
    "dmso": "dimethyl sulfoxide",
    "mct": "medium-chain triglycerides",
    "caprylic": "caprylic acid",
    "lauric": "lauric acid",
    "butyrate": "sodium butyrate",
    "pterostilbene": "pterostilbene",
    "apigenin": "apigenin",
    "fisetin": "fisetin",
    "luteolin": "luteolin",
    "baicalein": "baicalein",
    "berberine": "berberine hydrochloride",
    "milk-thistle": "silymarin",
    "olive-leaf": "oleuropein",

    # === DOSAGE UNITS ===
    "g": "gram",
    "kg": "kilogram",
    "ml": "milliliter",
    "l": "liter",
    "caps": "capsules",
    "tabs": "tablets",
    "soft": "softgels",
    "powder": "powder form",
    "liquid": "liquid form",
    "drops": "liquid drops",
    "spray": "oral spray",
    "patch": "transdermal patch",
    "lozenge": "sublingual lozenge",
    "gummy": "gummy supplement",

    # === DELIVERY FORMS ===
    "enteric": "enteric coated",
    "sr": "sustained release",
    "xr": "extended release",
    "tr": "time release",
    "dr": "delayed release",
    "buffered": "buffered form",
    "chelated": "chelated mineral",
    "micronized": "micronized form",
    "nano": "nano-particle",
    "liposomal": "liposomal encapsulated",
    "sublingual": "under tongue",
    "transdermal": "through skin",

    # === MEASUREMENT ABBREVIATIONS ===
    "ppm": "parts per million",
    "ppb": "parts per billion",
    "cfu": "colony forming units",
    "re": "retinol equivalents",
    "dfe": "dietary folate equivalents",
    "ate": "alpha-tocopherol equivalents",
    "rae": "retinol activity equivalents",
    "ne": "niacin equivalents",
    "dv": "daily value",
    "rda": "recommended dietary allowance",
    "ai": "adequate intake",
    "ul": "tolerable upper limit",
    "orac": "oxygen radical absorbance capacity",

    # === POPULAR BRAND ABBREVIATIONS ===
    "on": "optimum nutrition",
    "bsn": "bio-engineered supplements",
    "eas": "experimental applied sciences",
    "universal": "universal nutrition",
    "dymatize": "dymatize nutrition",
    "now": "now foods",
    "gnc": "general nutrition corporation",
    "musclepharm": "musclepharm corp",
    "cellucor": "cellucor supplements",
    "quest": "quest nutrition",
    "gaspari": "gaspari nutrition",
    "allmax": "allmax nutrition",
    "rsp": "rsp nutrition",
    "jym": "jym supplement science",
    "bnrg": "bnrg supplements",

    # === REGULATORY & TESTING ===
    "usp": "united states pharmacopeia",
    "nsf": "nsf international certified",
    "gmp": "good manufacturing practices",
    "organic": "usda organic certified",
    "non-gmo": "non-genetically modified",
    "gluten-free": "certified gluten-free",
    "vegan": "vegan-friendly",
    "kosher": "kosher certified",
    "halal": "halal certified",
    "third-party": "third-party tested",
    "coa": "certificate of analysis",
    "heavy-metals": "heavy metals tested",
    "purity": "purity tested",
    "potency": "potency verified"
}

# Additional context dictionaries for supplement categories
SUPPLEMENT_CATEGORIES = {
    "vitamins": ["vit a", "vit d", "vit d2", "vit d3", "vit e", "vit k", "vit k1", "vit k2",
                 "b1", "b2", "b3", "b5", "b6", "b7", "b9", "b12", "vit c", "folate", "biotin", "choline"],
    "minerals": ["ca", "mg-min", "fe", "zn", "cu", "mn", "se", "cr", "mo", "i", "k", "na", "p"],
    "amino_acids": ["arg", "gln", "gly", "tau", "try", "tyr", "phe", "met", "cys", "bcaa", "eaa"],
    "performance": ["crea", "cm", "pre", "pwo", "wpi", "wpc", "hmb", "beta-ala", "no"],
    "nootropics": ["nac", "cdp", "ps", "pc", "dmae", "gpc", "pqq", "nad", "nmn"],
    "adaptogens": ["ashwagandha", "rhodiola", "gin", "maca", "tribulus", "tongkat"],
    "antioxidants": ["coq10", "astaxanthin", "lycopene", "lutein", "resveratrol", "curc"],
    "sleep_recovery": ["mel", "gaba", "theanine", "mag-gly", "zma", "chamomile"],
    "joint_bone": ["gluc", "chond", "msm", "collagen", "cal-mag", "d3-k2"],
    "digestive": ["prob", "pre-bio", "digest", "betaine", "bromelain", "papain"],
    "immune": ["vit-c", "zinc-pic", "elderberry", "echi", "reishi", "beta-glucan"],
    "heart_circulation": ["hawthorne", "garlic", "red-yeast", "nattokinase"],
    "weight_management": ["caf", "green-coffee", "garcinia", "forskolin", "chromium-pic"]
}






In [26]:
EXTRA = {
    "omega3": "omega-3 fatty acids",
    "omega-3": "omega-3 fatty acids",
    "om3": "omega-3 fatty acids",
    "fishoil": "fish-oil (omega-3)",
    "fo": "fish-oil (omega-3)",
    "ashwa": "withania somnifera",
    "ashwag": "withania somnifera",
    "mag": "magnesium",
    "magcit": "magnesium citrate",
    "maggly": "magnesium glycinate",
    "bcomplex": "b-vitamin complex",
    "b-complex": "b-vitamin complex"
}
SUPP_ABBR_DICT.update(EXTRA)
#SUPP_ABBR_DICT.update(SUPPLEMENT_CATEGORIES)


In [27]:
# ── ADD (or merge) THESE NEW ENTRIES ───────────────────────────────
SUPP_ONLY_DICT= {

    # ── omega-3 & essential lipids ────────────────────────────────
    "fish oil":          "fish-oil (omega-3 fatty acids)",
    "fo":                "fish-oil (omega-3 fatty acids)",
    "omega 3":           "omega-3 fatty acids",
    "omega-3":           "omega-3 fatty acids",
    "omega3":            "omega-3 fatty acids",
    "om3":               "omega-3 fatty acids",
    "krill":             "krill-oil (omega-3)",
    "algal":             "algal-oil (vegetarian omega-3)",

    # ── vitamins (general & shorthand) ────────────────────────────
    "multi":             "multivitamin",
    "mv":                "multivitamin",
    "vit a":             "vitamin A (retinol)",
    "vit c":             "vitamin C (ascorbic acid)",
    "vc":                "vitamin C (ascorbic acid)",
    "vit d":             "vitamin D3 (cholecalciferol)",
    "vit d3":            "vitamin D3 (cholecalciferol)",
    "vd3":               "vitamin D3 (cholecalciferol)",
    "vit e":             "vitamin E (tocopherols)",
    "vit k2":            "vitamin K2 (menaquinone)",
    "k2":                "vitamin K2 (menaquinone)",
    "b complex":         "B-vitamin complex",
    "b-complex":         "B-vitamin complex",
    "bcomplex":          "B-vitamin complex",
    "b12":               "cobalamin (vitamin B12)",
    "b6":                "pyridoxine (vitamin B6)",
    "folic":             "folic acid (vitamin B9)",
    "folate":            "folate (vitamin B9)",

    # ── minerals & electrolytes ───────────────────────────────────
    "mag":               "magnesium",
    "mag gly":           "magnesium glycinate",
    "mag-gly":           "magnesium glycinate",
    "maggly":            "magnesium glycinate",
    "mag cit":           "magnesium citrate",
    "mag-cit":           "magnesium citrate",
    "mag mal":           "magnesium malate",
    "mag-mal":           "magnesium malate",
    "cal":               "calcium",
    "ca":                "calcium",
    "zn":                "zinc",
    "se":                "selenium",
    "k":                 "potassium",
    "pot":               "potassium",
    "na":                "sodium",
    "electrolytes":      "electrolyte blend",

    # ── proteins & amino blends ───────────────────────────────────
    "whey":              "whey protein concentrate",
    "wpi":               "whey protein isolate",
    "wpc":               "whey protein concentrate",
    "wph":               "whey protein hydrolysate",
    "iso":               "isolate protein powder",
    "casein":            "casein protein",
    "egg":               "egg-white protein",
    "pp":                "pea protein",
    "soy":               "soy protein",
    "hmb":               "beta-hydroxy beta-methylbutyrate",
    "beta ala":          "beta-alanine",
    "beta-ala":          "beta-alanine",
    "ba":                "beta-alanine",

    # ── performance & pre-workout staples ─────────────────────────
    "cre":               "creatine monohydrate",
    "crea":              "creatine monohydrate",
    "creatine":          "creatine monohydrate",
    "crt":               "creatine monohydrate",
    "pre":               "pre-workout supplement",
    "pwo":               "pre-workout supplement",
    "pump":              "nitric-oxide booster",
    "citr":              "l-citrulline",
    "citr mal":          "citrulline malate",
    "citr-mal":          "citrulline malate",
    "caf":               "caffeine anhydrous",
    "caff":              "caffeine anhydrous",
    "caffeine":          "caffeine anhydrous",
    "yoh":               "yohimbine",
    "yohimbe":           "yohimbine",

    # ── adaptogens & herbs ───────────────────────────────────────
    "ashwa":             "withania somnifera",
    "ashwag":            "withania somnifera",
    "rhodi":             "rhodiola rosea",
    "lion":              "lion’s-mane mushroom",
    "lionsmane":         "lion’s-mane mushroom",
    "cordy":             "cordyceps mushroom",
    "reishi":            "reishi mushroom",
    "turmeric":          "curcumin (curcuma longa)",
    "curc":              "curcumin (curcuma longa)",
    "egcg":              "epigallocatechin gallate",

    # ── nootropics & brain health ────────────────────────────────
    "gpc":               "alpha-GPC (glycerophosphocholine)",
    "agpc":              "alpha-GPC (glycerophosphocholine)",
    "theanine":          "l-theanine",
    "ltheanine":         "l-theanine",
    "tyrosine":          "l-tyrosine",
    "ltyr":              "l-tyrosine",
    "ps":                "phosphatidylserine",
    "pqq":               "pyrroloquinoline quinone",
    "nmn":               "nicotinamide mononucleotide",
    "nad+":              "nicotinamide adenine dinucleotide",

    # ── gut & immunity ───────────────────────────────────────────
    "probiotic":         "probiotic blend",
    "probiotics":        "probiotic blend",
    "synbiotic":         "synbiotic (pro- + prebiotic)",
    "fiber":             "dietary fibre supplement",
    "glucomannan":       "glucomannan fibre",
    "elderberry":        "sambucus nigra extract",
    "colostrum":         "bovine colostrum",

    # ── sleep / relaxation ───────────────────────────────────────
    "mel":               "melatonin",
    "melatonin":         "melatonin",
    "gaba":              "gamma-aminobutyric acid",
    "mag threonate":     "magnesium l-threonate",
    "mag-thr":           "magnesium l-threonate",

    # ── weight-management & metabolism ───────────────────────────
    "cla":               "conjugated linoleic acid",
    "green coffee":      "green-coffee bean extract",
    "gcb":               "green-coffee bean extract",
    "forskolin":         "coleus forskohlii extract",
    "gluco":             "glucomannan fibre",
    "chrom pic":         "chromium picolinate",

    # ── popular brands (shorthand used in chat) ──────────────────
    "on":                "Optimum Nutrition (brand)",
    "gnc":               "GNC (General Nutrition Corporation)",
    "myprotein":         "MyProtein (brand)",
    "now":               "NOW Foods (brand)",

}


### brands

In [28]:
SUPP_BRAND_CATEGORY = {
    # Staples – legacy science-minded houses
    "NOW Foods": "Staples",
    "Life Extension": "Staples",
    "Thorne Research": "Staples",
    "Pure Encapsulations": "Staples",
    "Designs for Health": "Staples",
    "Sports Research": "Staples",
    "Ortho Molecular": "Staples",
    "Metagenics": "Staples",

    # Mainstream value / mass-retail
    "Solgar": "Mass Retail",
    "Nature’s Way": "Mass Retail",
    "Swanson": "Mass Retail",
    "Solaray": "Mass Retail",
    "Nature Made": "Mass Retail",
    "Nature’s Bounty": "Mass Retail",
    "Spring Valley": "Mass Retail",
    "TruNature": "Mass Retail",
    "Kirkland Signature": "Mass Retail",

    # Budget bulk powders & single-ingredient caps
    "BulkSupplements": "Bulk Powders",
    "Nutricost": "Bulk Powders",
    "PureBulk": "Bulk Powders",

    # Clinical-dose sports / performance lines
    "Optimum Nutrition": "Sports Nutrition",
    "Transparent Labs": "Sports Nutrition",
    "Legion Athletics": "Sports Nutrition",
    "Kaged Muscle": "Sports Nutrition",
    "MyProtein": "Sports Nutrition",
    "Dymatize": "Sports Nutrition",
    "MuscleTech": "Sports Nutrition",

    # Omega-3 & specialty lipid leaders
    "Nordic Naturals": "Omega-3 / Lipids",
    "Carlson Labs": "Omega-3 / Lipids",
    "Viva Naturals": "Omega-3 / Lipids",

    # Botanical / integrative practitioners’ picks
    "Gaia Herbs": "Botanicals / Integrative",
    "Source Naturals": "Botanicals / Integrative",
    "Doctor’s Best": "Botanicals / Integrative",
    "Jarrow Formulas": "Botanicals / Integrative",
    "Double Wood Supplements": "Botanicals / Integrative",

    # Greens, collagen & lifestyle “daily stack” brands
    "Athletic Greens": "Daily Stack / Lifestyle",
    "AG1": "Daily Stack / Lifestyle",  # alias for Athletic Greens
    "Huel": "Daily Stack / Lifestyle",
    "Ritual": "Daily Stack / Lifestyle",
    "Vital Proteins": "Daily Stack / Lifestyle",
    "Garden of Life": "Daily Stack / Lifestyle",

    # Nootropic / biohacker favorites
    "Nootropics Depot": "Nootropics / Biohacking",
    "Tru Niagen": "Nootropics / Biohacking",
    "Real Mushrooms": "Nootropics / Biohacking"
}


### supps

In [29]:
df = pd.read_parquet("/content/drive/MyDrive/NLP/data/supplements_processed.parquet")

def expand_abbr(text, mapping=SUPP_ABBR_DICT):
    if not isinstance(text, str):
        return text
    out = text
    for abbr, full in mapping.items():
        # word-boundary replace, case-insensitive
        out = re.sub(rf"\b{re.escape(abbr)}\b", full, out, flags=re.I)
    return out

df["text_expanded"] = df["text"].fillna("").apply(expand_abbr)

### brands

In [30]:
from collections import Counter
import re

token_re = re.compile(r"[A-Za-z0-9\-+#']{2,}")

def corpus_terms(texts):
    for doc in texts:
        for t in token_re.findall(doc.lower()):
            yield t.strip("-")

seen = Counter(corpus_terms(df["text_expanded"]))
# How many times does each glossary key appear?
coverage = {k: seen[k] for k in SUPP_ABBR_DICT}
unused = [k for k,v in coverage.items() if v == 0]
missing = [t for t,c in seen.most_common(200) if t not in SUPP_ABBR_DICT and t not in unused]


### Build canonical lookup for both forms → short code

In [31]:
# maps any spelling (abbr OR full) → canonical short form
CANON = {}
for abbr, full in SUPP_ONLY_DICT.items():
    CANON[abbr.lower()] = abbr.lower()
    CANON[full.lower()] = abbr.lower()
for alias, canon in BRAND_ALIASES.items():
    BRAND_DICT[alias.lower()] = canon

### expanding text based on the dictionary

In [32]:
!pip install -q pyarrow           # full-featured, fastest

### Save the expnaded df. No need to run the above cells again.

In [33]:
# Save the updated DataFrame (with the new `text_expanded` column) to Parquet.
# ─────────────────────────────────────────────────────────────────────────────

# 1️⃣  Pick a descriptive output name so you don’t overwrite the source file
out_path = "/content/drive/MyDrive/NLP/data/supplements_processed_expanded.parquet"

# 2️⃣  Write it
df.to_parquet(
    out_path,
    engine="pyarrow",   # or "fastparquet" if that’s what you use
    index=False,        # don’t store the implicit 0-based index
    compression="snappy"  # fast, widely supported; change to "gzip", "brotli", etc. if you prefer
)

print(f"✓ DataFrame saved to {out_path}")


✓ DataFrame saved to /content/drive/MyDrive/NLP/data/supplements_processed_expanded.parquet


In [34]:
df = pd.read_parquet("/content/drive/MyDrive/NLP/data/supplements_processed_expanded.parquet")
print(df.head(50))

          type       id parent_id score  \
0   submission  1kv9962      None     1   
1   submission  1kv92e2      None     3   
2      comment  mu7mk2u   1kv92e2     1   
3      comment  mu7mtms   1kv92e2     1   
4      comment  mu7mp8w   1kv92e2     1   
5   submission  1kv8svt      None     1   
6   submission  1kv76td      None     1   
7      comment  mu78z17   1kv76td     2   
8      comment  mu7hzch   1kv76td     1   
9   submission  1kv74wt      None     2   
10     comment  mu7krfg   1kv74wt     1   
11  submission  1kv74my      None     0   
12  submission  1kv687a      None     2   
13     comment  mu6xpa0   1kv687a     5   
14     comment  mu745yc   1kv687a     1   
15     comment  mu6yii6   1kv687a     1   
16     comment  mu74r92   1kv687a     1   
17     comment  mu79bxm   1kv687a     1   
18     comment  mu70k45   1kv687a     1   
19     comment  mu7d6ep   1kv687a     1   
20  submission  1kv5rni      None     1   
21  submission  1kv4pkl      None     1   
22     comm

### 1 Inspect what columns you do have

In [35]:
print(df.columns.tolist())

['type', 'id', 'parent_id', 'score', 'title', 'upvote_ratio', 'comment_depth', 'text', 'author', 'created_utc', 'num_comments', 'flair', 'url', 'date_time', 'clean_text', 'lemmas', 'entities', 'text_expanded']


### spaCy EntityRuler

In [36]:
import spacy, re
from spacy.pipeline import EntityRuler
from tqdm.auto import tqdm

# patterns include BOTH forms but label will later be canonicalised
patterns = [{"label": "SUPPLEMENT", "pattern": term}
            for term in CANON.keys()]
#  brand patterns
patterns += [{"label": "BRAND", "pattern": pat}
             for pat in BRAND_DICT.keys()]

nlp = spacy.blank("en")
ruler = nlp.add_pipe("entity_ruler", config={"overwrite_ents": True})
ruler.add_patterns(patterns)




#make sure *something* sets sentence boundaries --------------
if "parser" not in nlp.pipe_names and "sentencizer" not in nlp.pipe_names \
   and "senter" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer")


def extract_canonical(text):
    """Return a list of short supplement codes found in text."""
    doc = nlp(text)
    found = set()
    for ent in doc.ents:
        key = CANON.get(ent.text.lower())
        if key:
            found.add(key)
    return list(found)

tqdm.pandas()
df["supplements"] = df["text_expanded"].fillna("").progress_apply(extract_canonical)


  0%|          | 0/8710 [00:00<?, ?it/s]

In [37]:
def extract_brands(text):
    doc = nlp(text)
    found = set()
    for ent in doc.ents:
        if ent.label_ == "BRAND":
            canon = BRAND_DICT.get(ent.text.lower())
            if canon:
                found.add(canon)
    return list(found)

df["brands"] = df["text_expanded"].fillna("").progress_apply(extract_brands)


  0%|          | 0/8710 [00:00<?, ?it/s]

###  Verify & clean the new column

In [38]:
out_path = "/content/drive/MyDrive/NLP/data/supplements_processed_expanded_spacyER.parquet"

# 2️⃣  Write it
df.to_parquet(
    out_path,
    engine="pyarrow",   # or "fastparquet" if that’s what you use
    index=False,        # don’t store the implicit 0-based index
    compression="snappy"  # fast, widely supported; change to "gzip", "brotli", etc. if you prefer
)

print(f"✓ DataFrame saved to {out_path}")

✓ DataFrame saved to /content/drive/MyDrive/NLP/data/supplements_processed_expanded_spacyER.parquet


In [39]:
df = pd.read_parquet("/content/drive/MyDrive/NLP/data/supplements_processed_expanded_spacyER.parquet")
print(df.columns.tolist())

['type', 'id', 'parent_id', 'score', 'title', 'upvote_ratio', 'comment_depth', 'text', 'author', 'created_utc', 'num_comments', 'flair', 'url', 'date_time', 'clean_text', 'lemmas', 'entities', 'text_expanded', 'supplements', 'brands']


In [40]:
print(df.columns.tolist())

['type', 'id', 'parent_id', 'score', 'title', 'upvote_ratio', 'comment_depth', 'text', 'author', 'created_utc', 'num_comments', 'flair', 'url', 'date_time', 'clean_text', 'lemmas', 'entities', 'text_expanded', 'supplements', 'brands']


In [41]:
df["supplements"].head(5)


,supplements
0,"[theanine, bcomplex, lion, b6, ashwag, rhodi, ..."
1,[ca]
2,[]
3,[]
4,[]


In [115]:
df["brands"].head(5)


,brands
0,[]
1,[Optimum Nutrition]
2,[]
3,[Optimum Nutrition]
4,[Optimum Nutrition]


In [116]:
df["supplements"].str.len().value_counts().head()

,count
supplements,
0,5397
1,2156
2,687
3,280
4,99


In [117]:
df["brands"].str.len().value_counts().head()

,count
brands,
0,6013
1,2361
2,330
3,6


In [46]:
df["created_datetime"] = pd.to_datetime(df["created_utc"].astype(float), unit="s", utc=True)

In [47]:
df["created_datetime"].head()

,created_datetime
0,2025-05-25 18:18:55+00:00
1,2025-05-25 18:10:55+00:00
2,2025-05-25 18:16:33+00:00
3,2025-05-25 18:17:56+00:00
4,2025-05-25 18:17:18+00:00


## Generate weak sentiment labels with VADER

In [42]:
nltk.download("vader_lexicon")

from nltk.sentiment import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [43]:
nltk.download("vader_lexicon")
sid = SentimentIntensityAnalyzer()

def sent_label(text):
    s = sid.polarity_scores(text)["compound"]
    return "positive" if s > 0.05 else "negative" if s < -0.05 else "neutral"

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Sentence-entity-sentiment extractor

In [44]:
def analyse_post(text, dt):
    """
    Yield rows: (supplement_short, sentiment_label, dt)
    Sentences with multiple supplements add one row per supplement.
    """
    rows = []
    doc = nlp(text)
    for sent in doc.sents:
        sent_sentiment = sent_label(sent.text)
        sups = {
            CANON.get(ent.text.lower())        # canonical short code
            for ent in sent.ents
            if ent.label_ == "SUPPLEMENT"
        }
        sups.discard(None)
        for s in sups:
            rows.append((s, sent_sentiment, dt))
    return rows

##  Run over the dataset (progress bar)

In [48]:
records = []
for _, row in tqdm(df[["text_expanded","created_datetime"]].iterrows(), total=len(df)):
    records.extend(analyse_post(row["text_expanded"], row["created_datetime"]))

sent_entity_df = pd.DataFrame(records, columns=["supplement","sentiment","dt"])
sent_entity_df.head()

  0%|          | 0/8710 [00:00<?, ?it/s]

,supplement,sentiment,dt
0,theanine,positive,2025-05-25 18:18:55+00:00
1,bcomplex,positive,2025-05-25 18:18:55+00:00
2,lion,positive,2025-05-25 18:18:55+00:00
3,b6,positive,2025-05-25 18:18:55+00:00
4,ashwag,positive,2025-05-25 18:18:55+00:00


##  Aggregate & Trend

In [49]:
supp_table = (
    sent_entity_df.groupby(["supplement","sentiment"]).size()
      .unstack(fill_value=0)
      .assign(total=lambda d: d.sum(axis=1))
      .sort_values("total", ascending=False)
)
supp_table.head(10)

sentiment,negative,neutral,positive,total
supplement,,,,
mag,190,332,537,1059
zn,113,182,230,525
creatine,107,30,372,509
theanine,68,113,227,408
b12,70,112,206,388
ashwag,49,73,160,282
ca,46,66,144,256
maggly,48,79,115,242
pot,38,62,117,217


### Monthly trend pivot (re-uses tz-aware pattern)

In [50]:
sent_entity_df["month"] = sent_entity_df["dt"].dt.to_period("M")
trend = (
    sent_entity_df.pivot_table(index="month",
                               columns=["supplement","sentiment"],
                               aggfunc="size",
                               fill_value=0)
)
trend.head()

supplement     agpc                    ashwag                  b complex  \
sentiment  negative neutral positive negative neutral positive  negative   
month                                                                      
2025-05           4      11       27       49      73      160         8   

supplement                       b12  ...     whey                       wpi  \
sentiment  neutral positive negative  ... negative neutral positive negative   
month                                 ...                                      
2025-05         12       31       70  ...        9      32       34        1   

supplement  yohimbe                        zn                   
sentiment  negative neutral positive negative neutral positive  
month                                                           
2025-05           5       1        3      113     182      230  

[1 rows x 168 columns]

##  Quick Plot for a supplement

### Normalise any user input before you look it up

In [51]:
! pip install rapidfuzz

In [52]:
from rapidfuzz import process, fuzz
import pandas as pd
import plotly.express as px

################################################################################
# 1) ───── Normaliser helper ───────────────────────────────────────────────────
################################################################################
CANONICAL_KEYS = list(SUPP_ONLY_DICT)          # just the keys, all lower-case

def normalise_query(q, scorer=fuzz.partial_ratio, score_cut=80):
    """
    Map a free-text query to one or more canonical supplement keys.
    Returns [] if nothing matches with confidence.
    """
    q = q.lower().strip().replace("-", "-").replace("–", "-")

    # 1. exact alias
    if q in SUPP_ONLY_DICT:
        return [q]

    # 2. fuzzy alias match (handles 'omega 3', 'fish oil', 'ashwa', etc.)
    match, score, _ = process.extractOne(q, CANONICAL_KEYS, scorer=scorer)
    if score >= score_cut:
        return [match]

    # 3. category name ('vitamins', 'nootropics', …)
    if q in SUPPLEMENT_CATEGORIES:
        return SUPPLEMENT_CATEGORIES[q]

    # 4. nothing recognised
    return []

################################################################################
# 2) ───── Sentiment plotter with normalisation ────────────────────────────────
################################################################################
def plot_sentiment(query):
    """
    Plot sentiment trend for a supplement alias *or* a category.
    * query – what the user types ('omega 3', 'b complex', 'vitamins', …)
    """
    keys = normalise_query(query)
    if not keys:
        raise ValueError(f"No supplement/category recognised for: {query!r}")

    # ─── single supplement ────────────────────────────────────────────────────
    if len(keys) == 1:
        key = keys[0]
        sub = trend[key]                            # DataFrame: pos/neg/neu
        title = f"Sentiment trend – {SUPP_ONLY_DICT.get(key, key).title()}"

    # ─── multiple (category) → average across members ────────────────────────
    else:
        # concatenate member frames side-by-side, then mean across members
        merged = pd.concat({k: trend[k] for k in keys}, axis=1)
        sub = merged.groupby(level=1, axis=1).mean()  # collapse member level
        title = f"Average sentiment – {query.title()} ({len(keys)} supplements)"

    # ensure DatetimeIndex
    if isinstance(sub.index, pd.PeriodIndex):
        sub.index = sub.index.to_timestamp()

    # plot
    fig = px.line(sub, title=title)
    fig.show()


In [53]:
plot_sentiment("creatine")

## Aspect-based sentiments

###  Install / set up (GPU-friendly)

In [54]:
# ------------------------------------------------------------------
# 0-A  Install core libs (spaCy & transformers will pull CUDA PyTorch automatically
#      on Colab/Kaggle if a GPU runtime is selected)
# ------------------------------------------------------------------
!pip -q install spacy tqdm "pyabsa==2.0.0"  # PyABSA optional
!python -m spacy download en_core_web_sm
# If you need SciSpaCy later:
# !pip -q install scispacy
# !pip -q install https://s3.amazonaws.com/allenai-scispacy/models/en_ner_bc5cdr_md-0.5.3.tar.gz

# ------------------------------------------------------------------
# 0-B  Enable spaCy GPU, if present
# ------------------------------------------------------------------


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 86.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [55]:
import spacy, torch
if torch.cuda.is_available():
    spacy.require_gpu()        # spaCy ops, incl. Transformer layers, move to CUDA
    DEVICE = 0
    print("✅  GPU detected – using CUDA")
else:
    DEVICE = -1
    print("⚠️  GPU not detected – running on CPU")

✅  GPU detected – using CUDA


###  GPU-backed sentence sentiment (RoBERTa)

In [56]:
from transformers import pipeline
sent_pipe = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    device=DEVICE                    # 0 if GPU, -1 if CPU
)

def sent_label(text):
    return sent_pipe(text[:512])[0]["label"].lower()   # positive/negative/neutral


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


###  Aspect regex + ABSA extraction

In [57]:
import re, itertools
ASPECTS = {
    "effectiveness": [r"\b(work|result|benefit|help)\w*\b"],
    "side_effect":   [r"\b(headache|nausea|insomnia|anxiety|side[- ]?effect)\b"],
    "taste":         [r"\b(taste|flavour|flavor|mix(es|ed)? well)\b"],
    "price":         [r"\b(price|cost|expensive|cheap)\b"],
}
ASPECT_PATTERNS = {k: re.compile("|".join(v), re.I) for k, v in ASPECTS.items()}

df = df.reset_index(drop=False).rename(columns={"index": "row_id"})  # ⇠ gives every row an ID

def absa_sentence(text, dt, supp_list, row_id):
    doc = nlp(text)
    rows = []
    for s in doc.sents:
        s_txt = s.text.lower()
        aspects = [a for a,p in ASPECT_PATTERNS.items() if p.search(s_txt)]
        if not aspects:
            continue
        pol = sent_label(s_txt)
        for sup in supp_list:
            for asp in aspects:
                rows.append((row_id, sup, asp, pol, dt))
    return rows

records = []
for rid, row in tqdm(df.iterrows(), total=len(df)):
    records.extend(absa_sentence(
        row["text_expanded"], row["created_datetime"],
        row["supplements"], row["row_id"]
    ))

absa_df = pd.DataFrame(records,
                       columns=["row_id", "supplement", "aspect",
                                "sentiment", "dt"])




  0%|          | 0/8710 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [58]:
out_path = "/content/drive/MyDrive/NLP/data/absa_df.parquet"

# 2️⃣  Write it
absa_df.to_parquet(
    out_path,
    engine="pyarrow",   # or "fastparquet" if that’s what you use
    index=False,        # don’t store the implicit 0-based index
    compression="snappy"  # fast, widely supported; change to "gzip", "brotli", etc. if you prefer
)

print(f"✓ DataFrame saved to {out_path}")

✓ DataFrame saved to /content/drive/MyDrive/NLP/data/absa_df.parquet


In [60]:
df = pd.read_parquet("/content/drive/MyDrive/NLP/data/absa_df.parquet")   # replace with your path
print(df.head(50))

    row_id supplement         aspect sentiment                        dt
0        0   theanine    side_effect   neutral 2025-05-25 18:18:55+00:00
1        0   bcomplex    side_effect   neutral 2025-05-25 18:18:55+00:00
2        0       lion    side_effect   neutral 2025-05-25 18:18:55+00:00
3        0         b6    side_effect   neutral 2025-05-25 18:18:55+00:00
4        0     ashwag    side_effect   neutral 2025-05-25 18:18:55+00:00
5        0      rhodi    side_effect   neutral 2025-05-25 18:18:55+00:00
6        0      folic    side_effect   neutral 2025-05-25 18:18:55+00:00
7        0        b12    side_effect   neutral 2025-05-25 18:18:55+00:00
8        0       gaba    side_effect   neutral 2025-05-25 18:18:55+00:00
9        0   theanine  effectiveness   neutral 2025-05-25 18:18:55+00:00
10       0   bcomplex  effectiveness   neutral 2025-05-25 18:18:55+00:00
11       0       lion  effectiveness   neutral 2025-05-25 18:18:55+00:00
12       0         b6  effectiveness   neutral 2025

## Utility: get top supplements & aspect ratings for a brand

In [71]:
def brand_summary(brand, top_n=10, min_mentions=5):
    """
    Parameters
    ----------
    brand : str
        Canonical brand name (e.g. "NOW Foods").  Aliases handled.
    top_n : int
        How many supplements to return (ranked by frequency).
    min_mentions : int
        Minimum #sentences needed before an aspect score is shown.
    """
    # map possible alias → canonical
    brand = BRAND_DICT.get(brand.lower(), brand)

    # —— rows that mention this brand ——————————————
    rows_b = df[df["brands"].apply(lambda L: brand in (L or []))]

    if rows_b.empty:
        raise ValueError(f"No rows found for brand '{brand}'")

    # —— 1.  most frequent supplements ——————————
    sup_counts = (rows_b.explode("supplements")
                           .value_counts("supplements")
                           .head(top_n))

    # —— 2.  aspect sentiment per supplement ——————
    id_set = set(rows_b.row_id)
    absa_b = absa_df[absa_df["row_id"].isin(id_set)]

    # map sentiment → score for quick mean calc
    SCORE = {"positive": 1, "neutral": 0, "negative": -1}

    records = []
    for sup in sup_counts.index:
        sub_df = absa_b[absa_b["supplement"] == sup]
        for asp in ASPECTS.keys():
            asp_df = sub_df[sub_df["aspect"] == asp]
            if len(asp_df) < min_mentions:
                continue
            mean = asp_df["sentiment"].map(SCORE).mean()
            records.append({
                "supplement": sup,
                "aspect": asp,
                "avg_sentiment": round(mean, 3),
                "n": len(asp_df)
            })

    result = (
        pd.DataFrame(records)
          .pivot(index="supplement", columns="aspect",
                 values="avg_sentiment")
          .reindex(sup_counts.index)           # keep top-N order
          .assign(total_mentions=sup_counts.values)
    )

    return result.sort_values("total_mentions", ascending=False)

# ── quick demo ─────────────────────────────────────────────
#brand_summary("NOW Foods").head()

KeyError: 'brands'

### Aggregate + plot

### 1 ️⃣ Quick sanity-check: counts

In [61]:
# How many rows per supplement?
absa_df["supplement"].value_counts().head()

# How many rows per (supplement, aspect) combo?
absa_df.groupby(["supplement","aspect"]).size().sort_values(ascending=False).head(10)


,,0
supplement,aspect,
mag,effectiveness,665
theanine,effectiveness,385
zn,effectiveness,295
ashwag,effectiveness,253
creatine,effectiveness,249
ca,effectiveness,225
b12,effectiveness,224
caffeine,effectiveness,193
mag,side_effect,181


### 2 ️⃣ Monthly trend pivot (one line of code)

In [62]:
absa_df["month"] = absa_df["dt"].dt.to_period("M")

trend = (
    absa_df.pivot_table(index="month",
                        columns=["supplement","aspect","sentiment"],
                        aggfunc="size",
                        fill_value=0)
)
trend.head()          # shows multi-index columns


supplement          agpc                                                \
aspect     effectiveness                    price          side_effect   
sentiment       negative neutral positive neutral positive    negative   
month                                                                    
2025-05                2      24       26       1        1           2   

supplement                ashwag                   ...            zn           \
aspect             effectiveness                   ... effectiveness    price   
sentiment  neutral      negative neutral positive  ...      positive negative   
month                                              ...                          
2025-05          7            19     121      113  ...            94        1   

supplement                                                                 \
aspect                      side_effect                     taste           
sentiment  neutral positive    negative neutral positive negative neutral   
month                                                                       
2025-05          6        1          31      29        6        6       5   

supplement           
aspect               
sentiment  positive  
month                
2025-05           2  

[1 rows x 436 columns]

### 3 ️⃣ Plot a sentiment-per-aspect trend for one supplement

In [63]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "colab"   # or "notebook", etc.

def plot_supp_aspect(supp: str, aspect: str, *, renderer=None):
    """
    Plot positive / neutral / negative counts by month for one
    (supplement, aspect) pair. Raises a clear error if the pair
    doesn’t exist and always returns the Figure object.
    """
    try:
        sub = trend.loc[:, (supp, aspect)]        # slice the 3 sentiment cols
    except KeyError:
        raise ValueError(f"No data for supplement={supp!r}, aspect={aspect!r}")

    # index is PeriodIndex → convert to timestamps so Plotly understands dates
    if hasattr(sub.index, "to_timestamp"):
        sub = sub.copy()          # avoid SettingWithCopy warning
        sub.index = sub.index.to_timestamp()

    fig = px.line(
        sub,
        markers=True,
        title=f"{supp.upper()} — {aspect} sentiment trend",
        labels={"value": "Count", "index": "Month"}
    )
    fig.update_layout(xaxis_title=None)
    fig.show(renderer=renderer)   # falls back to pio.renderers.default
    return fig


In [64]:
plot_supp_aspect("ashwag", "effectiveness")          # will display the chart

### 4 ️⃣ Stacked-bar: aspect mix for a supplement

In [65]:
import plotly.express as px
import plotly.io as pio

# Pick ONE that matches your notebook setup
pio.renderers.default = "colab"     # "notebook", "plotly_mimetype", "vscode", …

def stacked_aspect(supp: str, *, renderer: str | None = None):
    """
    One stacked bar per aspect showing pos / neu / neg counts
    for a single supplement, aggregated over all time.

    Returns the Figure object so you can further tweak it if you like.
    """
    # 1️⃣  Slice the data
    df_sub = absa_df.query("supplement == @supp")
    if df_sub.empty:
        raise ValueError(f"No rows found for supplement={supp!r}")

    # 2️⃣  Build a wide table: rows = aspect, columns = sentiment
    bar_tab = (
        df_sub.groupby(["aspect", "sentiment"], observed=True)
              .size()                               # count rows per (aspect, sentiment)
              .unstack(fill_value=0)                # wide form
              .reindex(columns=["positive", "neutral", "negative"],  # consistent order
                       fill_value=0)
              .reset_index()
    )

    # 3️⃣  Plot
    fig = px.bar(
        bar_tab,
        x="aspect",
        y=["positive", "neutral", "negative"],
        barmode="stack",
        title=f"{supp.upper()} — sentiment mix by aspect",
        labels={"value": "Count",
                "aspect": "Aspect",
                "variable": "Sentiment"}   # legend title
    )
    fig.show(renderer=renderer)            # falls back to pio.renderers.default
    return fig


In [79]:
stacked_aspect("ashwag")                     # uses global renderer

In [66]:
# build pivot: rows = aspect, cols = supplement, value = % positive
pivot = (
    absa_df.assign(pos=lambda d: (d["sentiment"]=="positive").astype(int))
        .pivot_table(index="aspect", columns="supplement",
                     values="pos", aggfunc="mean")
        .round(2)
)

fig = px.imshow(pivot,
                color_continuous_scale="RdYlGn",
                text_auto=True,
                aspect="auto",
                title="Share of POSITIVE sentences  (green = higher %)")
fig.update_coloraxes(colorbar_title="% positive")
fig.show()


In [ ]:
! pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.8 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/google/colab/_pip.py:86: ResourceWarning:

unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.11/dist-packages/pydeck-0.9.1.dist-info/top_level.txt' mode='r' encoding='utf-8'>

/usr/local/lib/python3.11/dist-packages/google/colab/_pip.py:86: ResourceWarning:

unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.11/dist-packages/streamlit-1.45.1.dist-info/top_level.txt' mode='r' encoding='utf-8'>

/usr/local/lib/python3.11/dist-packages/google/colab/_pip.py:86: ResourceWarning:

unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.11/dist-packages/watchdog-6.0.0.dist-info/top_level.txt' mode='r' encoding='utf-8'>



## Dashboard for local machine.

In [ ]:
%%writefile dashboard.py
import streamlit as st, plotly.express as px, pandas as pd

absa_df = pd.read_parquet("/content/drive/MyDrive/NLP/data/absa_df.parquet")   # save yours first!
absa_df["month"] = absa_df["dt"].dt.to_period("M")

st.title("Supplement Aspect Sentiment Dashboard")

supp = st.sidebar.selectbox("Supplement", sorted(absa_df["supplement"].unique()))
aspect = st.sidebar.selectbox("Aspect", sorted(absa_df["aspect"].unique()))

# trend line
trend = (
    absa_df.query("supplement == @supp and aspect == @aspect")
           .pivot_table(index="month", columns="sentiment", aggfunc="size", fill_value=0)
)
trend.index = trend.index.to_timestamp()
st.plotly_chart(px.line(trend, markers=True, title=f"{supp}-{aspect} trend"))

# aspect mix
mix = (
    absa_df[absa_df["supplement"] == supp]
          .groupby(["aspect","sentiment"]).size().unstack(fill_value=0).reset_index()
)
st.plotly_chart(px.bar(mix, x="aspect", y=["positive","neutral","negative"],
                       barmode="stack", title=f"{supp} – sentiment mix by aspect"))


Overwriting dashboard.py


In [ ]:
! streamlit run dashboard.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.99.74:8501

  Stopping...
  Stopping...
Exception ignored in atexit callback: <function shutdown at 0x7cd1650f1440>
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 2177, in shutdown
    for wr in reversed(handlerList[:]):
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 44, in signal_handler
    server.stop()
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/server/server.py", line 469, in stop
    self._runtime.stop()
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/runtime.py", line 324, in stop
    async_objs.eventloop.call_soon_threadsafe(stop_on_eventloop)
  File "/usr/lib/python3.11/asyncio/base_events.py", line 807, in call_soon_threadsafe
    self._check_closed()
  File "/usr/l

# Rough Work

##  Aggregate sentiment per supplement

In [ ]:
# 1) Explode each row so we have one row per (supplement, post)
# explode and aggregate
exploded = (
    df[["supplements", "sentiment", "created_datetime"]]
      .explode("supplements")
      .dropna(subset=["supplements"])
      .rename(columns={"supplements": "supplement", "created_datetime": "dt"})
)

sent_counts = (
    exploded.groupby(["supplement", "sentiment"]).size()
      .unstack(fill_value=0)
      .assign(total=lambda d: d.sum(1))
      .sort_values("total", ascending=False)
)

print(exploded.columns)          # <-- make sure 'dt' is in the list

print("min =", exploded["dt"].min(), "max =", exploded["dt"].max())
sent_counts.head(5)


Index(['supplement', 'sentiment', 'dt'], dtype='object')
min = 2020-01-14 02:31:00+00:00 max = 2020-11-04 23:49:00+00:00


sentiment,negative,neutral,positive,total
supplement,,,,
i,8322,4115,25064,37501
on,3562,1330,10183,15075
l,1484,916,5351,7751
no,1737,667,4218,6622
mg,939,674,3169,4782


##  Monthly sentiment trends

In [ ]:
# ────────────────────────────────────────────────────────────────────────────────
# 📅  BUILD A MONTHLY SENTIMENT TREND TABLE
# ────────────────────────────────────────────────────────────────────────────────
#
# 1️⃣  The `dt` column must already be timezone-aware pandas datetime64[ns, UTC]
#     If it isn’t, run:
#         df["dt"] = pd.to_datetime(df["created_utc"].astype(float), unit="s", utc=True)
#
# 2️⃣  We collapse each timestamp to the *calendar month* that contains it
#     via .dt.to_period("M").  Change the string if you need a different
#       – "W"  weekly buckets
#       – "Q"  quarterly  (2025Q1)
#       – "Y"  yearly
exploded["month"] = exploded["dt"].dt.to_period("Y")    # e.g. 2025-05

# 🚧  OPTIONAL FILTERS BEFORE PIVOT
#     Uncomment any of these to restrict the time range or the supplements.
# -------------------------------------------------------------------------
#exploded = exploded.query("dt >= '2025-05-27 18:18:55+00:00' and dt < '2025-05-6 18:18:55+00:00'")
#exploded = exploded[exploded["supplement"].isin(["msm","dhea","ashwagandha"])]

# 3️⃣  Create a pivot table where:
#       • rows   = each month bucket
#       • cols   = MultiIndex (supplement, sentiment)
#       • value  = count of rows
trend = (
    exploded
      .pivot_table(index="month",
                   columns=["supplement", "sentiment"],
                   aggfunc="size",     # simple counts
                   fill_value=0)       # missing combos → 0
)

print(exploded.columns)          # <-- make sure 'dt' is in the list

# 4️⃣  Inspect the first few rows
trend.head()



Index(['supplement', 'sentiment', 'dt', 'month'], dtype='object')


supplement    5-htp                     adra                ai          \
sentiment  negative neutral positive neutral positive negative neutral   
month                                                                    
2020             52      36      188       1        1        9       3   

supplement               ala          ... zinc-bis zinc-pic                   \
sentiment  positive negative neutral  ... positive negative neutral positive   
month                                 ...                                      
2020             19       35      14  ...        1       11      11       31   

supplement      zma                        zn                   
sentiment  negative neutral positive negative neutral positive  
month                                                           
2020             48      51      239      425     306     1332  

[1 rows x 773 columns]

In [ ]:
# ╔════════════════════════════════════════════════════════════════════════════╗
#   SENTIMENT-TREND PIVOT WITH SAFE DATE FILTER (tz-aware)                    #
# ╚════════════════════════════════════════════════════════════════════════════╝
#
# REQUIREMENTS
#   • exploded["dt"]  : pandas datetime64[ns, UTC]             (tz-aware ✅)
#   • exploded["sentiment"]   … and … exploded["supplement"]
#
# YOU CAN CHANGE --------------------------------------------------------------
START_STR = "2020-01-06"       # inclusive  (use UTC or leave as YYYY-MM-DD)
END_STR   = "2020-10-27"       # exclusive
#FOCUS_SUPPS = ["msm", "dhea", "ashwagandha"]    # leave None to keep all
FOCUS_SUPPS = None
# -----------------------------------------------------------------------------


# ── 1.  Build UTC-aware bounds so comparisons don’t raise “tz-naive vs tz-aware”
start = pd.Timestamp(START_STR, tz="UTC")
end   = pd.Timestamp(END_STR,   tz="UTC")

# ── 2.  Apply date-range mask
mask = (exploded["dt"] >= start) & (exploded["dt"] < end)
exploded_range = exploded.loc[mask].copy()

# ── 3.  Optional: keep only selected supplements
if FOCUS_SUPPS is not None:
    exploded_range = exploded_range[exploded_range["supplement"].isin(FOCUS_SUPPS)]

# ── 4.  Create a calendar-month bucket
exploded_range["month"] = exploded_range["dt"].dt.to_period("M")   # e.g. 2025-05

# ── 5.  Pivot: rows = month, columns = (supplement, sentiment), values = counts
trend = (
    exploded_range
      .pivot_table(index="month",
                   columns=["supplement", "sentiment"],
                   aggfunc="size",
                   fill_value=0)
)

#trend.head()      # peek at the resulting table

from IPython.display import display
display(trend.head())


supplement    5-htp                     adra                ai          \
sentiment  negative neutral positive neutral positive negative neutral   
month                                                                    
2020-01           3       0        9       0        0        0       0   
2020-02           1       0        9       0        0        2       0   
2020-03           3       0       18       1        1        0       1   
2020-04           3       6       20       0        0        0       1   
2020-05           4       1       13       0        0        0       0   

supplement               ala          ... zinc-bis zinc-pic                   \
sentiment  positive negative neutral  ... positive negative neutral positive   
month                                 ...                                      
2020-01           1        0       0  ...        0        0       1        2   
2020-02           2        3       1  ...        0        3       0        4   
2020-03           4        5       1  ...        0        2       2        6   
2020-04           1        3       2  ...        0        0       0        1   
2020-05           3        2       1  ...        0        0       2        1   

supplement      zma                        zn                   
sentiment  negative neutral positive negative neutral positive  
month                                                           
2020-01           1       4        8       10      17       31  
2020-02           6       3       43       33      21       99  
2020-03           6       3       15       60      37      153  
2020-04           7       5       23       37      34      121  
2020-05           2       8       27       36      31      155  

[5 rows x 770 columns]

## interpret / use trend

In [ ]:
import plotly.express as px

def plot_supp_trend(supp):
    wide = trend.loc[:, supp]            # quick slice: sentiment columns only
    wide.index = wide.index.to_timestamp()
    fig = px.line(wide, title=f"Sentiment trend for {supp}", markers=True)
    fig.show()

plot_supp_trend("ashwagandha")